# Hands-On 3: Parallelization with MPI

Welcome to Hands-on _Parallelization with MPI_. This Hands-on comprises 3 sessions. Next table shows the documents and
files needed to develop each one of the exercises.

|  Sessions     | Codes               | files              | 
| --------------| --------------------| ------------------ |
| Session 1     |  Basic Operations   |   operations.c   | 
| Session 2     | Algebraic Function  |  function.c      | 
| Session 3     |  Tridiagonal Matrix |   tridiagonal.c  | 


## `Basic Operations`

In [1]:
%%writefile ./material/operations.c
#include <mpi.h>
#include <stdio.h> 
#define SIZE 12

int main (int argc, char **argv){
  int i, sum = 0, subtraction = 0, mult = 1;
  int array[SIZE];

  char operations[] = {'+', '-', '*'};
  char operationsRec;
  int numberOfProcessors, id, to, from, tag = 1000;
  int result, value;
  
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numberOfProcessors);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Status status;

  switch(id){
    case 0: //Master
      for(i = 0; i < SIZE; i++){
        array[i] = i + 1;
        printf("%d: %d\t", i, array[i]);
      }

      printf("\n");
      for(to = 1; to < numberOfProcessors; to++) {
        MPI_Send(&array, SIZE, MPI_INT, to, tag, MPI_COMM_WORLD);
        MPI_Send(&operations[to-1], 1, MPI_CHAR, to, tag, MPI_COMM_WORLD);
      }
      break;
    default: // Workers
      MPI_Recv(&array, SIZE, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);
      MPI_Recv(&operationsRec, 1, MPI_CHAR, 0, tag, MPI_COMM_WORLD, &status);
      switch (operationsRec) {
        case '+': // Worker 1
          value = 0; 
          for(i = 0; i < SIZE; i++)
          value += array[i];
          break;
        case '-': // Worker 2
          value = 0;
          for(i = 0; i < SIZE; i++)
          value -= array[i];
          break;
        case '*': // Worker 3
          value = 1;
          for(i = 0; i < SIZE; i++)
          value *= array[i];
          break;
      }
  }

  MPI_Send(&value, 1, MPI_INT, 0, tag, MPI_COMM_WORLD);
  MPI_Send(&operationsRec, 1, MPI_CHAR, 0, tag, MPI_COMM_WORLD);

  for(to = 1; to < numberOfProcessors; to++) {
    MPI_Recv(&result, 1, MPI_INT, to, tag, MPI_COMM_WORLD, &status);
    MPI_Recv(&operationsRec, 1, MPI_CHAR, to, tag, MPI_COMM_WORLD, &status);
    printf ("(%c) = %d\n", operationsRec, result);
  }
  
  // Validating the results
  for(i = 0; i < SIZE; i++) 
  array[i] = i + 1;

  for(i = 0; i < SIZE; i++)
    printf("array[%d] = %d\n", i, array[i]);

  for(i = 0; i < SIZE; i++) 
  {
    sum = sum + array[i];
    subtraction = subtraction - array[i]; 
    mult = mult * array[i];
  }

  printf("Sum = %d\n", sum); 
  printf("Subtraction = %d\n", subtraction); 
  printf("Multiply = %d\n", mult);

  return 0;

}

Overwriting ./material/operations.c


### Run the Code

In [2]:
!mpicc ./material/operations.c -o operations

In [3]:
!mpirun -np 4 ./operations

0: 1	1: 2	2: 3	3: 4	4: 5	5: 6	6: 7	7: 8	8: 9	9: 10	10: 11	11: 12	
(+) = 78
(-) = -78
(*) = 479001600
array[0] = 1
array[1] = 2
array[2] = 3
array[3] = 4
array[4] = 5
array[5] = 6
array[6] = 7
array[7] = 8
array[8] = 9
array[9] = 10
array[10] = 11
array[11] = 12
Sum = 78
Subtraction = -78
Multiply = 479001600


## `Algebraic Function`

The idea of this Hands-on is to make an algorithm that uses the
`MPI_Recv` and `MPI_Send` routines in the Master-Worker Paradigm in such
a way that in the sequential code:

In [4]:
%%writefile ./material/function.c
#include <stdio.h>
#include <mpi.h>

int main (int argc, char **argv){
    double coef[4], result[4], x = 10, total, received_result;   
    int tag = 1000, numberOfProcessors, id, to, from, i, id_indexed, received_index;

    for(i = 1; i <= 4; i++) coef[i-1] = i;

    MPI_Init(&argc, &argv);
    MPI_Comm_size(MPI_COMM_WORLD, &numberOfProcessors);
    MPI_Comm_rank(MPI_COMM_WORLD, &id);
    MPI_Status status;

    switch(id){
        case 0: //Master
            for(to = 1; to < numberOfProcessors; to++) {
                MPI_Send(&x, 1, MPI_DOUBLE, to, tag, MPI_COMM_WORLD);
            }
            break;

        default: // Workers
            MPI_Recv(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD, &status);
            
            id_indexed = 3 - id;
            result[id_indexed] = coef[id_indexed];
            for(i = 1; i <= id; i++)
                result[id_indexed] *= x;
            break;
    }

    MPI_Send(&result[id_indexed], 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
    MPI_Send(&id_indexed, 1, MPI_INT, 0, tag, MPI_COMM_WORLD);

    if(id == 0) {
        for(from = 1; from < numberOfProcessors; from++) {
            MPI_Recv(&received_result, 1, MPI_DOUBLE, from, tag, MPI_COMM_WORLD, &status);
            MPI_Recv(&received_index, 1, MPI_INT, from, tag, MPI_COMM_WORLD, &status);
            result[received_index] = received_result;
            printf ("(%d) = %lf\n", received_index, result[received_index]);
            total += received_result;
        }

        if(total > 0) printf("Total: %.5lf\n", total);
    }

    MPI_Finalize();
    return 0;
}


Overwriting ./material/function.c


### Run the Code

In [5]:
!mpicc ./material/function.c -o ./output/function

In [10]:
!mpirun -np 4 ./output/function

(2) = 30.000000
(1) = 200.000000
(0) = 1000.000000
Total: 1230.00000


## `Tridiagonal Matrix`

In [7]:
%%writefile ./material/tridiagonal.c
#include <stdio.h>
#include <mpi.h>
#define ORDER 4

void printMatrix (int m[][ORDER]) 
{
  int i, j;
  for(i = 0; i < ORDER; i++) {
    printf ("| ");
    for (j = 0; j < ORDER; j++) {
      printf ("%3d ", m[i][j]);
    }
    printf ("|\n");
  }
  printf ("\n");
}

int main (int argc, char **argv){
  int k[3] = {100, 200, 300};
  int matrix[ORDER][ORDER] = {0}, received_matrix[ORDER][ORDER], i, j;
  
  int tag = 1000, numberOfProcessors, id;

  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &numberOfProcessors);
  MPI_Comm_rank(MPI_COMM_WORLD, &id);
  MPI_Status status;

  if(id == 0) {
    for(int to = 1; to < numberOfProcessors; to++) {
      MPI_Send(&matrix, ORDER * ORDER, MPI_INT, to, tag, MPI_COMM_WORLD);
    }
    for(i = 1; i < numberOfProcessors; i++){
      MPI_Recv(&received_matrix, ORDER * ORDER, MPI_INT, MPI_ANY_SOURCE, tag, MPI_COMM_WORLD, &status);
      int worker_id = status.MPI_SOURCE;
      switch(worker_id){
        case 1:
          for(j = 0; j < ORDER; j++) matrix[j][j]     += received_matrix[j][j];    //main diagonal           
          break;
        case 2:
          for(j = 0; j < ORDER; j++) {
            matrix[j + 1][j] += received_matrix[j + 1][j];    //subdiagonal
            matrix[j][j + 1] += received_matrix[j][j + 1];    //superdiagonal
          }
          break;
        case 3:
          for(j = 0; j < ORDER; j++) {
            if(!(j == (ORDER - 1))) matrix[j][j + 1] += received_matrix[j][j + 1];    //superdiagonal
            if(!(j == 0)) matrix[j][j - 1] += received_matrix[j][j - 1];    //subdiagonal
          }
          break;
      }
    }
  } else {
    MPI_Recv(&matrix, ORDER * ORDER, MPI_INT, 0, tag, MPI_COMM_WORLD, &status);
    switch(id){
      case 1:
        for(i = 0; i < ORDER; i++)
          for(j = 0; j < ORDER; j++) if( i == j ) matrix[i][j] = i + j + 1 + k[0];             
        break;
      case 2:
        for(i = 0; i < ORDER; i++)
          for(j = 0; j < ORDER; j++) if(i == (j + 1)){
            matrix[i][j] = i +  j + 1 + k[1];
            matrix[j][i] = matrix[i][j] + k[2];
          }
        break;
      case 3:
        for(i = 0; i < ORDER; i++)
          for(j = 0; j < ORDER; j++) if(!(i == j) && !(i == (j + 1))) matrix[i][j] = 0;
        break;
    }
    MPI_Send(&matrix, ORDER * ORDER, MPI_INT, 0, tag, MPI_COMM_WORLD);
  }

  printf("ID: %d\n", id);
  printMatrix(matrix);

  MPI_Finalize();
  return 0;
}


Overwriting ./material/tridiagonal.c


### Run the Code

In [8]:
!mpicc ./material/tridiagonal.c -o ./output/tridiagonal 

In [9]:
!mpirun -np 4 ./output/tridiagonal

ID: 1
| 101   0   0   0 |
|   0 103   0   0 |
|   0   0 105   0 |
|   0   0   0 107 |

ID: 2
|   0 502   0   0 |
| 202   0 504   0 |
|   0 204   0 506 |
|   0   0 206   0 |

ID: 3
|   0   0   0   0 |
|   0   0   0   0 |
|   0   0   0   0 |
|   0   0   0   0 |

ID: 0
| 101 502   0   0 |
| 202 103 504   0 |
|   0 204 105 506 |
|   0   0 206 107 |



## References

M. Boratto. Hands-On Supercomputing with Parallel Computing. Available: https://github.com/muriloboratto/Hands-On-Supercomputing-with-Parallel-Computing. 2022.

B. Chapman, G. Jost and R. Pas. Using OpenMP: Portable Shared Memory Parallel Programming. The MIT Press, 2007, USA.